In [ ]:
# @title 1. Instalar librerías necesarias
!pip install -q -U google-generativeai # Instala/Actualiza solo google-generativeai

In [1]:
# @title 2. Importar librerías y configurar API Key
import re
import time
import pandas as pd
import csv # Necesario para quoting en to_csv
import google.generativeai as genai
import json
import os
from google.colab import userdata # Para manejar secrets en Colab

# --- Configuración de la API de Gemini ---
# Nota: Debes guardar tu API key en los 'Secrets' de Colab con el nombre 'GOOGLE_API_KEY'
# Si no usas secrets, descomenta la siguiente línea y pega tu key (menos seguro)
# GOOGLE_API_KEY="TU_API_KEY_AQUI"
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key de Gemini configurada correctamente desde Secrets.")
except userdata.SecretNotFoundError:
    print("Advertencia: Secret 'GOOGLE_API_KEY' no encontrado.")
    print("Por favor, añade tu API key a los 'Secrets' de Colab (Panel izquierdo -> Icono de llave).")
    GOOGLE_API_KEY = None
except Exception as e:
    print(f"Ocurrió un error al configurar la API Key: {e}")
    GOOGLE_API_KEY = None

# Configuración del modelo Gemini a usar
# Puedes cambiar a "gemini-1.5-flash" si prefieres velocidad/menor coste
# Experimental = gemini-2.5-pro-exp-03-25
model_name = "gemini-1.5-flash" # o "gemini-1.5-pro-latest" u otro disponible

if GOOGLE_API_KEY:
    model = genai.GenerativeModel(model_name)
    print(f"Modelo Gemini '{model_name}' inicializado.")
else:
    model = None
    print("El modelo Gemini no pudo ser inicializado por falta de API Key.")

API Key de Gemini configurada correctamente desde Secrets.
Modelo Gemini 'gemini-1.5-flash' inicializado.


In [2]:
# @title 3. Definir la Lista de Frases a Analizar

frases_para_analizar = [
    "Esta enfermedad de amanecer nostálgico\nEn la oscuridad de madrugada sórdida\nTanto tiempo andando en piloto automático\nNavegando pa' no pisar tierra sólida\n\nCreo que he recordado esta mañana mágica\nTardes de tu mano en ese puerto trágico\nDonde tu mirada era la Luna única\nEn nuestro desierto de absoluta oscuridad\n\nAstronauta dónde anda tu nave de cristal\nDonde construimos un hogar galáctico\nAcaso surcando el universo práctico\nO en otro planeta de la vía láctea\n\nMi llorar es un asunto sintomático\nDe mi propia nave que anda a toda máquina\nDeshaciéndose entre numerosos cánticos\nDisparando a las murallas de la humanidad\n\nEsta enfermedad de amanecer nostálgico\nQue no se me pasa ni con la felicidad\nMe tiene añorando algún contacto tántrico\nCon esa mirada de infinita locura\n\nCreo que he recordado esta mañana trágica\nTardes de tu mano en ese puerto mágico\nDonde las palabras eran mero trámite\nDe nuestros lenguajes fuera de la lógica\n\nAstronauta dónde fue tu nave de cristal\nDonde construimos un hogar galáctico\nAcaso surcando el universo práctico\nO en otro planeta de la vía láctea"

    #"Me gustaría cortarle el pelo, ya está muy largo",
    # "Necesito ir al banco.",
    #"Él vino tarde y trajo el vino que prometió.",
    #"La copia del informe está lista; el estudiante copia descaradamente.",
    #"Bajo la intensa lluvia, bajo al refugio rápidamente.",
    # "Vi al hombre con el telescopio.",
    #"Compramos pasteles y bebidas frías para la fiesta.",
    #"El perro persiguió al gato hasta que se cansó.",
    #"El sol brilla fuerte y los niños juegan alegres en el parque.",
    # "El libro que me recomendaste ayer tiene una trama muy compleja.",
    #"¡Caramba!, no esperaba encontrarte aquí.",
    # "Forbes: los dueños de Chile entre los más ricos del mundo",
    #"Me va a costar un ojo de la cara",
    # "pucha vecino, no me quedan más bolsitas",
    #"Tía no hay no como que eso pepino nada tortuga y sandía realmente elementales de un pájaro" # Frase agramatical/random
    # "Los corazones palpitan y palpitan",
    #"Para conectarme con el alma",
    # "Abrazar mi inmensa soledad en el destello púrpura del blur nocturno",
    # "El frío gris de la sangre que llevo dentro",
    # "Messi es lo más grande del mundo",
    # "Cuántas copas tenés!?",
    #"El organismo acogió los requerimientos de la oposición que señalaban que la socióloga de 80 años infringió la Carta Fundamental al participar de la fallida compraventa al Estado de la propiedad de su familia en calle Guardia Vieja.",
    #"El ministro dio un discurso en la banca.",  # Ambigüedad: asiento vs. sector financiero
    #"Pásame la lima del escritorio.",  # Ambigüedad: fruta vs. herramienta
    #"Vi a la mujer con el sombrero rojo.",  # Ambigüedad sintáctica
    #"Necesito reparar el ordenador.",  # Dialectalismo (ES) vs "computadora" (LATAM)
    #"Ese reloj vale un ojo de la cara.",  # Modismo
    #"El paciente tiene un caso grave de caso.",  # Homónimos
    #"Se me ha muerto el móvil otra vez.",  # Coloquialismo (ES)
    #"El cura la cura con fe.",  # Sustantivo vs verbo
    #"¿Vos tenés frío o calor?",  # Voseo rioplatense
    #"Hazme un like en el post.",  # Neologismo digital
    #"Habían muchas personas en la sala.",  # Error común (debería ser "había")
    #"El niño come caramelos y su hermano también.",  # Elipsis
    #"Correr rápido él parque ayer.",  # Agramatical
    #"La jirafa alta del zoológico duerme de pie.",  # Subordinación
    #"¡Ché, qué quilombo este trámite!",  # Lunfardo argentino
    #"El influencer subió un unboxing del último iPhone.",  # Anglicismo
    #"Tiré de la cadena y se rompió.",  # Polisemia (cadena de baño vs collar)
    #"El tiempo vuela sin mirar atrás.",  # Lenguaje figurado
    #"Si hubieras venido, lo habrías visto.",  # Condicional compuesto
    #"La data está corrupta en el sistema.",  # Calco del inglés
    #"El equipo de desarrollo debugueó el código.",  # Verbo adaptado del inglés
    #"Sus palabras eran dagas afiladas.",  # Metáfora
    #"El concierto estuvo brutal, hermano.",  # Coloquialismo juvenil
    #"El test dio un falso positivo.",  # Técnico/medicalizado
    #"¿Cómo que no sabías eso?",  # Interrogativa retórica
    #"El autobús se llenó de pibes ruidosos.",  # Registro informal (ES)
    #"La app crasheó al subir el archivo pesado.",  # Mix español-inglés

   # --- AÑADE MÁS FRASES AQUÍ SI QUIERES ---


    #"El científico, quien había dedicado décadas a estudiar los efectos del cambio climático en los ecosistemas marinos, publicó un informe alarmante que, sin embargo, fue ignorado por las autoridades, a pesar de las advertencias de la comunidad internacional.",
    #"Si hubieras llegado a tiempo, como te pedí repetidamente por mensaje, y no te hubieras quedado atrapado en el tráfico, que según dijiste fue causado por un accidente, habríamos podido evitar esta situación incómoda que ahora nos obliga a replantear todo el proyecto.",
    #"La novela, escrita en un estilo que mezcla el realismo mágico con la crítica social, y ambientada en un pueblo donde el tiempo parece haberse detenido, narra la historia de tres generaciones de mujeres que luchan contra las convenciones de una sociedad profundamente machista.",
    #"Aunque me encantaría acompañarte al concierto, cuyo cartel anuncia a varias bandas que admiro desde la adolescencia, debo priorizar la entrega del informe que mi jefe, bastante exigente por cierto, me ha pedido para mañana a primera hora.",
    #"Los estudiantes, algunos de los cuales habían viajado desde provincias lejanas para asistir al seminario, expresaron su frustración cuando, tras horas de espera en un auditorio mal ventilado, se anunció que el ponente principal había cancelado su participación por motivos de salud.",
    #"Creo firmemente que, de haber contado con los recursos adecuados y el apoyo necesario, que lastimosamente nunca llegaron, nuestro equipo habría podido desarrollar una solución innovadora para este problema que afecta a millones de personas en zonas rurales.",
    #"El chef, tras haber trabajado en los mejores restaurantes de París y Tokio, donde perfeccionó técnicas que luego fusionaría con ingredientes autóctonos, decidió abrir un pequeño local en su ciudad natal, la cual nunca dejó de amar a pesar de sus constantes viajes.",
    #"La noche, con su manto de estrellas temblorosas que parecen susurrar secretos ancestrales, envolvía la ciudad dormida donde los sueños y las pesadillas se fundían en un baile eterno bajo la mirada indiferente de la luna.",
    #"Entre las grietas del tiempo, donde habitan los recuerdos olvidados y las promesas rotas, encontré las palabras que jamás te dije, aquellas que se quedaron suspendidas como hojas secas en el aire estático de un otoño perpetuo.",
    #"La democracia, ese frágil entramado de consensos y disensos que se teje entre el clamor de las calles y el silencio de los despachos, requiere de ciudadanos no sólo informados sino profundamente comprometidos con la defensa de valores que trascienden intereses particulares.",
    #"Para realizar la configuración óptima del dispositivo, asegúrese de verificar que los puertos de entrada, los cuales suelen ubicarse en la parte posterior según el modelo adquirido, estén correctamente alineados con los conectores del cable HDMI, evitando así posibles fallos de transmisión de datos durante el proceso de sincronización.",
    #"Oye, ¿te acuerdas de ese tipo que conocimos en la fiesta de Ana, el que llevaba una camisa hawaiana y no paraba de hablar de sus viajes por Asia? Pues resulta que trabaja en la misma empresa que mi hermana, y me dijo que si quería mandarle el CV, pero la verdad es que ni siquiera sé si están contratando ahora mismo.",
    #"El correr rápido los árboles mientras el sol brillaba noche las estrellas cantaban canciones de verde melancolía bajo un mar de nubes rugientes",
    #"El CEO, tipo super ocupado con su jet privado y sus reuniones en la nube, me soltó un 'feedback' diciendo que hay que 'pivotear la estrategia para optimizar el customer journey', pero yo flipé porque ni él mismo se cree esa retórica corporativa.",
    #"Entre 1790 y 1800 en Francia, los districts fueron el primer nivel de subdivisión de los départements, y fueron posteriormente reemplazados por los arrondissements",
    #"El seseo es una característica fonético-fonológica de muchas variedades de las lenguas española y gallega consistente en la existencia de un único fonema fricativo coronal que es pronunciado como una fricativa alveolar sorda [s]",

    #"La actuación en el Teatro Épico necesitaba que los actores interpretasen sus personajes de manera convincente sin tratar de convencer ni a la audiencia ni a ellos mismos de que eran en realidad los personajes que interpretaban",
    #"un presentimiento aún más negro carcomía el corazón de los filósofos: que a las personas simplemente les disgustaba la idea de ser libres y que, dados los sinsabores que el ejercicio de su libertad podía implicar, rechazaban la perspectiva de su emancipación"

    # Ambigüedad y polisemia
    #"El banco cerró temprano, así que no pude sentarme.",
    #"El juez dictó la sentencia con dureza.",
    #"La luz verde indica que puedes avanzar.",

    # Modismos y coloquialismos
    #"Estoy hasta la coronilla con este taco",
    #"Ese examen estuvo facilito",
    #"No me des el avión, dime la verdad.",

    # Registro formal e informal
    #"Se informa a los pasajeros que el vuelo ha sido cancelado debido a condiciones meteorológicas adversas.",
    #"Oye, bro, ¿qué onda con el vacile de esta noche?",

    # Dialectalismos
    #"Voy a la guagua, ya es tarde.",
    #"Se me antojó una sopaipilla",
    #"¿Me pasás la birra?",

    # Frases técnicas o especializadas
    #"El algoritmo de backpropagation ajusta los pesos de la red neuronal mediante la minimización del error.",
    #"El colapso de la función de onda ocurre al realizar una medición en un sistema cuántico.",
    #"La fotosíntesis convierte la energía solar en energía química en forma de glucosa",

    # Figuras retóricas y expresiones poéticas
    #"La nostalgia es un río que arrastra los recuerdos.",
    #"Las estrellas titilaban como ojos curiosos en la noche infinita.",
    #"El viento susurraba secretos entre las hojas de los árboles.",

    # Lenguaje de redes sociales y tecnología
    #"El tweet se hizo viral en cuestión de minutos.",
    #"Me ghosteó después de tres días de hablar sin parar.",
    #"El servidor se cayó justo cuando iba a hacer la compra online.",

    # Errores comunes y fenómenos lingüísticos
    #"Ayer habían muchas personas en el parque.",
    #"Me se olvidó tu nombre.",
    #"Tuvo que de volver el libro.",

    # Frases complejas con subordinación
    #"Aunque llovía torrencialmente y la carretera estaba resbaladiza, decidimos seguir nuestro camino, convencidos de que llegaríamos a tiempo para la reunión, aunque, en el fondo, sabíamos que era poco probable.",
    #"El libro que compré ayer, el cual fue recomendado por mi profesor de literatura, trata sobre la influencia del realismo mágico en la narrativa latinoamericana contemporánea.",

    # Neologismos y préstamos lingüísticos
    #"Voy a hacer un streaming en Twitch esta noche.",
    #"Necesito actualizar el software del router para mejorar el performance de la red.",
    #"El metaverso promete cambiar la forma en que interactuamos digitalmente."

    # Ambigüedad y polisemia
    #"Vi a mi amigo con los prismáticos.",  # ¿Quién tiene los prismáticos?
    #"Juan vio a Pedro cansado.",  # ¿Quién está cansado?
    #"La llave de la caja está rota.",  # ¿Llave física o clave de acceso?
    #"No pude asistir a la presentación porque estaba enfermo.",  # ¿Quién estaba enfermo?
    #"La batería está baja.",  # ¿Instrumento musical o fuente de energía?

    # Modismos y expresiones idiomáticas
    #"Echar agua al mar.",  # Hacer algo inútil
    #"Le dieron gato por liebre.",  # Engañar a alguien
    #"Estar en las nubes.",  # Despistado o soñador
    #"Poner manos a la obra.",  # Empezar a trabajar
    #"Más vale tarde que nunca.",  # Refrán

    # Dialectalismos y variantes regionales
    #"Me pilló el atasco y llegué tarde.",  # España, "atasco" en vez de "tráfico"
    #"Ayer se descompuso mi carro.",  # México, "carro" en vez de "coche"
    #"Se me rompió el móvil otra vez.",  # España, "móvil" en vez de "celular"
    #"Agarré el colectivo para ir al centro.",  # Argentina, "colectivo" en vez de "autobús"
    #"Voy a la tienda a comprar un refresco.",  # México, "refresco" en vez de "gaseosa"

    # Lenguaje técnico y especializado
    #"El modelo de machine learning sobreajustó los datos de entrenamiento.",
    #"El código tiene una fuga de memoria en la función recursiva.",
    #"El ADN mitocondrial solo se hereda por vía materna.",
    #"El principio de incertidumbre impide conocer con precisión la posición y velocidad de una partícula.",
    #"El protocolo HTTPS cifra la comunicación entre el cliente y el servidor.",

    # Lenguaje coloquial y juvenil
    #"No me rayes, tío, que estoy hasta arriba de trabajo.",  # España
    #"Este finde vamos a pegar alto carrete.",  # Chile
    #"Qué facha tenés hoy, amigo!",  # Argentina, "facha" como apariencia
    #"Se armó el desmadre en la fiesta anoche.",  # México
    #"Este juego es una locura, no puedo dejar de jugar.",  # Expresión juvenil

    # Neologismos y términos digitales
    #"Voy a hacer un streaming en Twitch esta noche.",
    #"El video se hizo viral en TikTok.",
    #"La inteligencia artificial generativa está revolucionando la industria.",
    #"Me cancelaron en Twitter por mi último comentario.",
    #"El CEO anunció que van a pivotar el negocio hacia el e-commerce.",

    # Frases complejas con subordinación
    #"Aunque tenía sueño, decidió seguir estudiando porque el examen era al día siguiente.",
    #"El profesor, quien había ganado varios premios, presentó su nuevo libro en la universidad.",
    #"Si hubieras llegado más temprano, habríamos alcanzado el último tren.",
    #"El informe que enviaste ayer, el cual revisé esta mañana, tiene algunos errores de formato.",
    #"Los clientes que se registraron en la preventa recibirán un descuento especial.",

    # Lenguaje figurado y poético
    #"El silencio gritaba más que las palabras.",
    #"Los recuerdos se desvanecen como huellas en la arena.",
    #"El mar susurraba historias antiguas al oído del viento.",
    #"Las sombras bailaban al compás de la luna.",
    #"Sus palabras eran puñales que se clavaban en el alma.",

    # Errores comunes y fenómenos lingüísticos
    #"Ayer habían muchas personas en el parque.",  # Error de concordancia
    #"Me se olvidó tu nombre.",  # Leísmo / loísmo / laísmo
    #"Andé por todo el centro buscando una tienda abierta.",  # Uso incorrecto de pretérito
    #"Dijistes que ibas a venir temprano.",  # Uso incorrecto del pretérito
    #"Si yo sabría que iba a llover, habría traído paraguas.",  # Error en condicional

    # Frases absurdas o sin sentido
    #"El elefante volador dibujó un círculo en el cielo con su trompa.",
    #"Las montañas suspiraban de alivio cuando cayó la primera nieve.",
    #"Un reloj sin tiempo marca la eternidad.",
    #"La sombra del pez nadaba sobre la luna reflejada en el agua.",
    #"Los colores de la música pintaban el aire de azul y rojo.",

    # Preguntas retóricas
    #"¿Cuántas veces tengo que decirlo?",
    #"¿Acaso crees que esto es un juego?",
    #"¿Quién en su sano juicio haría algo así?",
    #"¿No es obvio lo que está pasando?",
    #"¿Cómo puede algo tan simple ser tan complicado?"
]

print(f"Se analizarán {len(frases_para_analizar)} frases.")

Se analizarán 1 frases.


In [12]:
# @title 4. Procesamiento Principal (Generación de CoT y JSON en bucle) - CORREGIDO V2

# --- Definiciones (Schemas, Mapeo, Parseo - Asegúrate que estén aquí o en una celda ANTERIOR ejecutada) ---
# ... (el código de schemas, category_map, parse_cot va aquí, SIN CAMBIOS) ...
schemas = {
    "sustantivo": {
      "categoria": "sustantivo", "palabra_analizada": None, "lemma": None, "genero": None, "numero": None, "tipo": None, "diminutivo_comun": None, "aumentativo_comun": None, "definicion_contextual": None, "definicion_general": None, "ejemplos_uso": []
    },
    "adjetivo": {
      "categoria": "adjetivo", "palabra_analizada": None, "lemma": None, "genero": None, "numero": None, "grado": None, "apocope": None, "modifica_a": None, "definicion_general": None, "ejemplos_uso": []
    },
    "verbo": {
      "categoria": "verbo", "palabra_analizada": None, "infinitivo": None, "modo": None, "tiempo": None, "persona": None, "numero": None, "participio": None, "gerundio": None, "transitividad": None, "definicion_general": None, "ejemplos_uso": []
    },
    "determinante": {
      "categoria": "determinante", "palabra_analizada": None, "tipo": None, "subtipo": None, "genero": None, "numero": None, "persona": None, "distancia": None, "definicion_funcion": None, "ejemplos_uso": []
    },
    "pronombre": {
      "categoria": "pronombre", "palabra_analizada": None, "tipo": None, "persona": None, "genero": None, "numero": None, "caso": None, "tonicidad": None, "referente_aproximado": None, "definicion_funcion": None, "ejemplos_uso": []
    },
    "adverbio": {
      "categoria": "adverbio", "palabra_analizada": None, "lemma": None, "tipo": None, "grado": None, "modifica_a": None, "definicion_general": None, "ejemplos_uso": []
    },
    "preposicion": {
      "categoria": "preposición", "palabra_analizada": None, "tipo": None, "usos_comunes": [], "definicion_funcion": "Introduce un complemento indicando una relación específica.", "ejemplos_uso": []
    },
    "conjuncion": {
      "categoria": "conjunción", "palabra_analizada": None, "tipo": None, "subtipo": None, "definicion_funcion": "Une palabras, sintagmas u oraciones.", "ejemplos_uso": []
    },
    "interjeccion": {
      "categoria": "interjección", "palabra_analizada": None, "tipo": None, "subtipo": None, "emocion_tipica": None, "definicion_funcion": "Expresa emociones súbitas o llama la atención.", "ejemplos_uso": []
    },
    "puntuacion": {
        "categoria": "puntuacion", "palabra_analizada": None, "tipo": None, "funcion": None
    },
    "otro": {
        "categoria": "otro", "palabra_analizada": None, "descripcion": "Categoría no identificada o palabra desconocida."
    }
}
category_map = {
    "sustantivo": "sustantivo", "noun": "sustantivo",
    "adjetivo": "adjetivo", "adj": "adjetivo",
    "verbo": "verbo", "verb": "verbo", "aux": "verbo",
    "determinante": "determinante", "det": "determinante",
    "artículo": "determinante",
    "pronombre": "pronombre", "pron": "pronombre",
    "adverbio": "adverbio", "adv": "adverbio",
    "preposición": "preposicion", "adp": "preposicion",
    "conjunción": "conjuncion", "conj": "conjuncion", "cconj": "conjuncion", "sconj": "conjuncion",
    "interjección": "interjeccion", "intj": "interjeccion",
    "puntuación": "puntuacion", "punct": "puntuacion",
    "numeral": "determinante", "num": "determinante",
    "partícula": "otro", "part": "otro",
    "símbolo": "otro", "sym": "otro",
    "contracción": "otro",
    "otro": "otro", "x": "otro"
}
def parse_cot(cot_text):
    pattern = re.compile(r"\*\s*'([^']+)'\s*:\s*.*?\|\s*(\b[\wáéíóúüñ]+\b)\s*\|?.*", re.IGNORECASE | re.MULTILINE)
    pattern_alt = re.compile(r"\*\s*'([^']+)'\s*:\s*(\b[\wáéíóúüñ]+\b)", re.IGNORECASE | re.MULTILINE)
    step1_text = ""
    if cot_text:
        step1_match = re.search(r"\*\*1\. Análisis por Palabra:\*\*(.*?)\*\*2\. Estructura Básica:\*\*", cot_text, re.DOTALL | re.IGNORECASE)
        if step1_match: step1_text = step1_match.group(1)
        else:
             step1_match_alt = re.search(r"\*\*1\. Análisis por Palabra:\*\*(.*?)\*\*3\. Reflexión Profunda", cot_text, re.DOTALL | re.IGNORECASE)
             if step1_match_alt: step1_text = step1_match_alt.group(1)
             elif cot_text.strip().startswith("**1. Análisis por Palabra:**"): step1_text = cot_text.split("**1. Análisis por Palabra:**", 1)[1]
    parsed_words = []
    if step1_text:
        matches_step1 = pattern.findall(step1_text)
        if not matches_step1: matches_step1 = pattern_alt.findall(step1_text)
        for i, match in enumerate(matches_step1):
            word = match[0].strip()
            category_raw = match[1].strip().lower()
            category_clean = category_raw.split()[0]
            category_key = category_map.get(category_clean, "otro")
            parsed_words.append({"index": i, "text": word, "category_raw": category_raw, "category_key": category_key})
    return parsed_words
# --- Fin Definiciones ---

resultados = [] # Reiniciar resultados para esta ejecución

if not model:
    print("ERROR: El modelo LLM no está inicializado. Verifica la API Key.")
else:
    if 'frases_para_analizar' not in locals():
         print("ERROR: La lista 'frases_para_analizar' no está definida. Ejecuta la celda 3.")
    else:
        print(f"Iniciando procesamiento de {len(frases_para_analizar)} frases...")
        # --- BUCLE PRINCIPAL ---
        for index, frase_ejemplo in enumerate(frases_para_analizar):
            print(f"\n--- Procesando frase {index + 1}/{len(frases_para_analizar)}: '{frase_ejemplo[:100]}...' ---") # Acortar print de frase larga
            complex_cot_limpio = "Error: CoT no generada"
            respuesta_json_string = json.dumps({"error": "JSON no generado"}, ensure_ascii=False)
            parsed_data = [] # Inicializar parsed_data para cada frase

            # --- 4.1 Generar Cadena de Pensamiento (CoT) ---
            try:
                prompt_cot = f"""
Analiza la siguiente frase/texto en español de forma detallada, perspicaz y estructurada: '{frase_ejemplo}'

Genera una "Cadena de Pensamiento" (CoT) siguiendo este formato exacto en TRES pasos:

**1. Análisis por Palabra:**
(Indica: Palabra | Posible Ambigüedad Contextual (Sí/No/Nota MUY breve) | Categoría Gramatical Asignada | Justificación MUY Breve *solo si hubo ambigüedad y requiere aclaración mínima aquí*)
*   'Palabra1': [Ambigüedad] | [Categoría Asignada] | [Justificación si aplica]
*   ... (continúa para todas las palabras si el texto es corto. Si es largo, analiza las 15-20 palabras más relevantes o estructuralmente importantes).

**2. Estructura Básica:**
*   Tipo de oración/texto: [Simple/Compuesta/Compleja/Párrafo/Estrofa...]
*   Componentes principales o ideas clave identificadas: [Sujeto/Verbo/Complementos o Temas/Estructura general]

**3. Reflexión Profunda y Síntesis (Forma, Contexto, Significado):**
(Sigue este orden. **Sé conciso donde no haya nada relevante.** Enfoca el detalle en los puntos interesantes.)
*   **3.1. Ambigüedades Formales Significativas:** Discute **únicamente** si existen ambigüedades léxicas, morfológicas, sintácticas o referenciales **relevantes o interesantes**. Clasifícalas y explica cómo se resuelven o por qué persisten. Si no hay, indica: "Ninguna ambigüedad formal relevante."
*   **3.2. Contexto Inferido (Tipo de Texto, Registro, Intención):** Infiere tipo de texto (literario/poético, periodístico, coloquial, etc.) y registro (formal/informal...). Justifica. Considera intención pragmática (afirmar, preguntar, ironía, metáfora...). *Si es literario/poético*, identifica recursos estilísticos (metáfora, símil, aliteración...). ¿Cómo influye el contexto/intención en ambigüedades (si las hubo en 3.1)?
*   **3.3. Significado e Interpretación:** Describe significado literal (resumen). Analiza significado implícito (presuposiciones/implicaturas) considerando el contexto (3.2). Explica posibles interpretaciones globales si las ambigüedades/contexto/implicaturas lo permiten. Si es unívoco, indícalo.
*   **3.4. Conclusión Sintética:** Resume complejidad, claridad/ambigüedad y aspectos destacables (forma-contexto-significado).

**Instrucciones Adicionales:**
*   Pasos 1 y 2: Concisos. Para textos largos en Paso 1, enfócate en palabras clave.
*   Paso 3: Detallado **solo en puntos relevantes**. Usa "Ninguna relevante" si aplica.
*   Usa categorías gramaticales estándar (o español consistente).
"""
                generation_config_cot = genai.types.GenerationConfig(temperature=0.3)
                print("   Generando CoT...")
                response_cot = model.generate_content(prompt_cot, generation_config=generation_config_cot)
                complex_cot_raw = response_cot.text

                start_index = complex_cot_raw.find("**1. Análisis por Palabra:**")
                if start_index != -1:
                    complex_cot_limpio = complex_cot_raw[start_index:].strip()
                else:
                    complex_cot_limpio = complex_cot_raw.strip()
                    print("   Advertencia: No se encontró el inicio esperado en la CoT. Usando CoT cruda.")

                # --- 4.2 Generar Respuesta JSON (LÓGICA MODIFICADA) ---
                print("   Intentando parsear CoT para guiar generación JSON...")
                parsed_data = parse_cot(complex_cot_limpio) # Intentar parsear

                # -------- INICIO MODIFICACIÓN --------
                json_prompt_generated = False
                if parsed_data: # SI el parseo del Paso 1 FUNCIONÓ (lista no vacía)
                    print("   Parseo de Paso 1 exitoso. Generando JSON basado en palabras listadas.")
                    # Construir el prompt JSON ORIGINAL (basado en la lista parsed_data)
                    prompt_parts_json = [f"Frase original a analizar: '{frase_ejemplo}'"]
                    prompt_parts_json.append("\n--- Cadena de Pensamiento Previa (Usar como Contexto Principal): ---")
                    prompt_parts_json.append(complex_cot_limpio)
                    prompt_parts_json.append("--- Fin de Cadena de Pensamiento Previa ---")
                    prompt_parts_json.append("\n--- Tarea Principal ---")
                    prompt_parts_json.append("Basándote en la frase original Y en la Cadena de Pensamiento anterior:")
                    # *** Instrucción clave original ***
                    prompt_parts_json.append("Genera un análisis lingüístico detallado para CADA palabra listada en el 'Paso 1' de la CoT.")
                    prompt_parts_json.append("Para cada palabra, produce un objeto JSON siguiendo el schema correspondiente a su categoría gramatical asignada.")
                    # ... (resto del prompt original: schemas, instrucciones, formato esperado) ...
                    prompt_parts_json.append("\n--- Schemas JSON por Categoría (Rellena estos campos): ---")
                    for cat_key, schema_template in schemas.items():
                        prompt_parts_json.append(f"\nSchema para '{cat_key.upper()}':")
                        schema_str = json.dumps(schema_template, indent=2, ensure_ascii=False)
                        prompt_parts_json.append(schema_str)
                    prompt_parts_json.append("\n--- Instrucciones Específicas (¡IMPORTANTE!) ---")
                    prompt_parts_json.append("*   Utiliza la categoría asignada en el 'Paso 1' de la CoT para seleccionar el schema JSON correcto.")
                    prompt_parts_json.append("*   **CRUCIAL: Al rellenar los campos del JSON, CONSIDERA DETENIDAMENTE las reflexiones hechas en el 'Paso 3' de la CoT.**")
                    prompt_parts_json.append("*   Si el Paso 3 discutió desambiguaciones, asegúrate de que el JSON lo refleje.")
                    prompt_parts_json.append("*   Rellena 'palabra_analizada' con la forma exacta de la palabra.")
                    prompt_parts_json.append("*   Infiere 'lemma' y otros rasgos morfológicos.")
                    prompt_parts_json.append("*   Proporciona definiciones claras y 1-2 ejemplos de uso nuevos.")
                    prompt_parts_json.append("*   Si un campo no aplica o no se puede determinar, déjalo como `null` u omítelo.")
                    prompt_parts_json.append("\n--- Formato de Salida Esperado ---")
                    prompt_parts_json.append("Genera UN ÚNICO objeto JSON como respuesta. Este objeto debe ser una lista (`[]`) donde cada elemento es un diccionario JSON correspondiente a una palabra del Paso 1 de la CoT.")
                    final_prompt_json = "\n".join(prompt_parts_json)
                    json_prompt_generated = True

                else: # SI el parseo del Paso 1 FALLÓ (lista vacía)
                    print("   Parseo de Paso 1 falló o incompleto. Generando JSON basado en CoT completa.")
                    # Construir el prompt JSON ALTERNATIVO (más general)
                    prompt_parts_json_alt = [f"Frase/Texto original a analizar: '{frase_ejemplo}'"]
                    prompt_parts_json_alt.append("\n--- Cadena de Pensamiento Previa (Usar como Contexto Principal): ---")
                    prompt_parts_json_alt.append(complex_cot_limpio) # CoT completa como contexto
                    prompt_parts_json_alt.append("--- Fin de Cadena de Pensamiento Previa ---")
                    prompt_parts_json_alt.append("\n--- Tarea Principal (Alternativa) ---")
                    prompt_parts_json_alt.append("Basándote en el texto original Y en la Cadena de Pensamiento completa proporcionada:")
                    # *** Instrucción clave alternativa ***
                    prompt_parts_json_alt.append("Genera un análisis lingüístico detallado en formato JSON para las PALABRAS o CONCEPTOS CLAVE identificados y discutidos en la CoT (especialmente en los Pasos 1 y 3, si el Paso 1 fue incompleto).")
                    prompt_parts_json_alt.append("Para cada palabra/concepto clave relevante, produce un objeto JSON siguiendo el schema apropiado para su categoría gramatical inferida.")
                    # ... (resto del prompt: schemas, instrucciones generales, formato esperado) ...
                    prompt_parts_json_alt.append("\n--- Schemas JSON por Categoría (Rellena estos campos): ---")
                    for cat_key, schema_template in schemas.items():
                        prompt_parts_json_alt.append(f"\nSchema para '{cat_key.upper()}':")
                        schema_str = json.dumps(schema_template, indent=2, ensure_ascii=False)
                        prompt_parts_json_alt.append(schema_str)
                    prompt_parts_json_alt.append("\n--- Instrucciones Específicas (¡IMPORTANTE!) ---")
                    prompt_parts_json_alt.append("*   Infiere la categoría gramatical de cada palabra/concepto clave seleccionado para elegir el schema JSON correcto.")
                    prompt_parts_json_alt.append("*   **CRUCIAL: Al rellenar los campos del JSON, CONSIDERA DETENIDAMENTE las reflexiones hechas en el 'Paso 3' de la CoT.**")
                    prompt_parts_json_alt.append("*   Si el Paso 3 discutió desambiguaciones, asegúrate de que el JSON lo refleje.")
                    prompt_parts_json_alt.append("*   Rellena 'palabra_analizada' con la palabra o concepto clave.")
                    prompt_parts_json_alt.append("*   Infiere 'lemma' y otros rasgos morfológicos relevantes si aplica.")
                    prompt_parts_json_alt.append("*   Proporciona definiciones claras y 1-2 ejemplos de uso nuevos.")
                    prompt_parts_json_alt.append("*   Si un campo no aplica o no se puede determinar, déjalo como `null` u omítelo.")
                    prompt_parts_json_alt.append("\n--- Formato de Salida Esperado ---")
                    prompt_parts_json_alt.append("Genera UN ÚNICO objeto JSON como respuesta. Este objeto debe ser una lista (`[]`) donde cada elemento es un diccionario JSON correspondiente a una palabra/concepto clave relevante analizado.")
                    final_prompt_json = "\n".join(prompt_parts_json_alt)
                    json_prompt_generated = True

                # ----- Ejecutar la llamada para generar JSON (si se generó algún prompt) -----
                if json_prompt_generated:
                    generation_config_json = genai.types.GenerationConfig(temperature=0.2)
                    print("   Generando JSON...")
                    response_json = model.generate_content(final_prompt_json, generation_config=generation_config_json)
                    raw_response_text_json = response_json.text
                    cleaned_json_text = re.sub(r"^\s*```json\s*|\s*```\s*$", "", raw_response_text_json, flags=re.MULTILINE | re.DOTALL).strip()

                    try:
                        parsed_json_object = json.loads(cleaned_json_text)
                        respuesta_json_string = json.dumps(parsed_json_object, indent=2, ensure_ascii=False)
                        print("   JSON generado y validado correctamente.")
                    except json.JSONDecodeError as json_err:
                        print(f"   Error: La respuesta del LLM para JSON no es válida: {json_err}")
                        error_output = {"error": "La respuesta del LLM no pudo ser parseada como JSON.","detalle": str(json_err),"respuesta_recibida": raw_response_text_json}
                        respuesta_json_string = json.dumps(error_output, indent=2, ensure_ascii=False)
                else:
                    # Esto no debería ocurrir con la lógica actual, pero por si acaso
                    print("   Error: No se pudo generar un prompt para el JSON.")
                    respuesta_json_string = json.dumps({"error": "No se pudo generar el prompt para JSON."}, ensure_ascii=False)

                # -------- FIN MODIFICACIÓN --------

            except Exception as e:
                print(f"   Error general procesando la frase: {e}")
                if complex_cot_limpio == "Error: CoT no generada": complex_cot_limpio = f"Error al generar CoT: {e}"
                if respuesta_json_string == json.dumps({"error": "JSON no generado"}, ensure_ascii=False): respuesta_json_string = json.dumps({"error": f"Error durante el procesamiento general: {e}"}, ensure_ascii=False)

            # --- 4.3 Guardar Resultados ---
            resultados.append({
                "frase": frase_ejemplo,
                "Complex CoT": complex_cot_limpio,
                "respuesta": respuesta_json_string
            })

            # --- Pausa ---
            print("   ...pausa...")
            time.sleep(2)

        # --- FIN BUCLE PRINCIPAL ---
        print(f"\nProcesamiento completado para {len(resultados)} frases en esta ejecución.")

Iniciando procesamiento de 1 frases...

--- Procesando frase 1/1: 'Esta enfermedad de amanecer nostálgico
En la oscuridad de madrugada sórdida
Tanto tiempo andando en ...' ---
   Generando CoT...
   Intentando parsear CoT para guiar generación JSON...
   Parseo de Paso 1 falló o incompleto. Generando JSON basado en CoT completa.
   Generando JSON...
   JSON generado y validado correctamente.
   ...pausa...

Procesamiento completado para 1 frases en esta ejecución.


In [13]:
# @title 5. Exportar Resultados a CSV (Añade si ya existe)

# Asegúrate de que 'os' y 'csv' estén importados (deberían estarlo desde la Celda 2)
import os
import csv # Necesario para la constante quoting

if not resultados:
    print("No se generaron nuevos resultados en esta ejecución para añadir/exportar.")
else:
    print(f"\nCreando DataFrame con {len(resultados)} nuevos resultados...")
    # Crear DataFrame con los nuevos resultados de esta ejecución
    df_nuevos_resultados = pd.DataFrame(resultados, columns=['frase', 'Complex CoT', 'respuesta'])

    # Mostrar las primeras filas de los *nuevos* datos como verificación
    print("\nPrimeras 5 filas de los NUEVOS resultados a añadir/exportar:")
    print(df_nuevos_resultados.head())

    # Nombre del archivo de salida
    nombre_archivo_csv = "diccionario_linguistico_output.csv"

    try:
        # --- LÓGICA PARA AÑADIR O CREAR ---
        if os.path.exists(nombre_archivo_csv):
            # Si el archivo EXISTE, añade los nuevos datos SIN la cabecera
            print(f"\nArchivo '{nombre_archivo_csv}' encontrado. Añadiendo {len(df_nuevos_resultados)} nuevas filas...")
            df_nuevos_resultados.to_csv(
                nombre_archivo_csv,
                mode='a',          # 'a' para append (añadir)
                header=False,       # No escribir la cabecera de nuevo
                index=False,
                encoding='utf-8-sig',
                quoting=csv.QUOTE_NONNUMERIC
            )
            print("Nuevas filas añadidas correctamente.")
        else:
            # Si el archivo NO EXISTE, créalo CON la cabecera
            print(f"\nArchivo '{nombre_archivo_csv}' no encontrado. Creando nuevo archivo...")
            df_nuevos_resultados.to_csv(
                nombre_archivo_csv,
                mode='w',          # 'w' para write (escribir - modo por defecto)
                header=True,        # Escribir la cabecera al crear
                index=False,
                encoding='utf-8-sig',
                quoting=csv.QUOTE_NONNUMERIC
            )
            print(f"Archivo '{nombre_archivo_csv}' creado correctamente.")

        # --- Mensaje final ---
        print(f"\nResultados guardados/añadidos en '{nombre_archivo_csv}'")
        print("Puedes encontrar el archivo en el panel izquierdo de Colab (icono de carpeta).")

        # Opcional: Descargar el archivo (puede ser útil después de añadir varias veces)
        # from google.colab import files
        # files.download(nombre_archivo_csv)

    except Exception as e:
        print(f"\nError durante la escritura/añadido al CSV: {e}")


Creando DataFrame con 1 nuevos resultados...

Primeras 5 filas de los NUEVOS resultados a añadir/exportar:
                                               frase  \
0  Esta enfermedad de amanecer nostálgico\nEn la ...   

                                         Complex CoT  \
0  **1. Análisis por Palabra:**\n\n* 'Enfermedad'...   

                                           respuesta  
0  [\n  {\n    "categoria": "sustantivo",\n    "p...  

Archivo 'caso_ejemplo.csv' no encontrado. Creando nuevo archivo...
Archivo 'caso_ejemplo.csv' creado correctamente.

Resultados guardados/añadidos en 'caso_ejemplo.csv'
Puedes encontrar el archivo en el panel izquierdo de Colab (icono de carpeta).


In [15]:
# @title 6. Cargar y Mostrar Fila Específica del CSV

import pandas as pd
import os
import json # Para pretty-print del JSON

# --- Parámetros ---
#@markdown Introduce el número de fila que quieres ver (empezando desde 0):
numero_fila_a_mostrar = 0 #@param {type:"integer"}
# --- Fin Parámetros ---

# Nombre del archivo CSV (debe coincidir con el usado en la celda anterior)
nombre_archivo_csv = "diccionario_linguistico_output.csv"

# Variable para almacenar el DataFrame
df = None

# --- Cargar el archivo CSV (si existe) ---
if not os.path.exists(nombre_archivo_csv):
    print(f"Error: El archivo '{nombre_archivo_csv}' no existe aún.")
    print("Asegúrate de haber ejecutado las celdas anteriores para generarlo.")
else:
    try:
        print(f"Cargando datos desde '{nombre_archivo_csv}'...")
        # Leer el CSV. Pandas suele manejar bien las comillas si se usó QUOTE_NONNUMERIC.
        df = pd.read_csv(nombre_archivo_csv)
        print(f"Archivo cargado con {len(df)} filas.")
    except Exception as e:
        print(f"Error al cargar el archivo CSV: {e}")
        df = None # Asegurarse de que df es None si falla la carga

# --- Mostrar la fila seleccionada (si el DataFrame se cargó) ---
if df is not None:
    if df.empty:
        print("El archivo CSV está vacío.")
    # Validar el número de fila introducido
    elif numero_fila_a_mostrar < 0 or numero_fila_a_mostrar >= len(df):
        print(f"Error: Número de fila inválido ({numero_fila_a_mostrar}).")
        print(f"Por favor, introduce un número entre 0 y {len(df) - 1}.")
    else:
        try:
            # Obtener la fila usando .iloc (indexación basada en enteros)
            fila_seleccionada = df.iloc[numero_fila_a_mostrar]

            print(f"\n--- Mostrando Fila {numero_fila_a_mostrar} ---")

            # 1. Imprimir la Frase
            print("\n[1] Frase (Input):")
            print("-" * 20)
            print(fila_seleccionada['frase'])

            # 2. Imprimir la Complex CoT
            print("\n[2] Complex CoT (Output 0 - Markdown String):")
            print("-" * 40)
            print(fila_seleccionada['Complex CoT']) # Se imprime como string Markdown

            # 3. Imprimir la Respuesta (JSON pretty-printed)
            print("\n[3] Respuesta (Output - JSON String Pretty-Printed):")
            print("-" * 50)
            respuesta_str = fila_seleccionada['respuesta']
            try:
                # Intentar parsear y re-formatear el JSON para legibilidad
                respuesta_obj = json.loads(respuesta_str)
                print(json.dumps(respuesta_obj, indent=2, ensure_ascii=False))
            except json.JSONDecodeError:
                # Si no es un JSON válido (podría ser un mensaje de error guardado como string)
                print("Error al parsear la 'respuesta' como JSON. Mostrando como texto:")
                print(respuesta_str)
            except TypeError:
                 # Manejar caso donde 'respuesta' podría ser NaN u otro tipo no-string
                 print("El valor en la columna 'respuesta' no es un string JSON válido. Mostrando valor crudo:")
                 print(respuesta_str)


            print(f"\n--- Fin Fila {numero_fila_a_mostrar} ---")

        except KeyError as ke:
            print(f"Error: Falta una columna esperada en el CSV: {ke}")
        except Exception as e:
            print(f"Error inesperado al mostrar la fila: {e}")

Cargando datos desde 'caso_ejemplo.csv'...
Archivo cargado con 1 filas.

--- Mostrando Fila 0 ---

[1] Frase (Input):
--------------------
Esta enfermedad de amanecer nostálgico
En la oscuridad de madrugada sórdida
Tanto tiempo andando en piloto automático
Navegando pa' no pisar tierra sólida

Creo que he recordado esta mañana mágica
Tardes de tu mano en ese puerto trágico
Donde tu mirada era la Luna única
En nuestro desierto de absoluta oscuridad

Astronauta dónde anda tu nave de cristal
Donde construimos un hogar galáctico
Acaso surcando el universo práctico
O en otro planeta de la vía láctea

Mi llorar es un asunto sintomático
De mi propia nave que anda a toda máquina
Deshaciéndose entre numerosos cánticos
Disparando a las murallas de la humanidad

Esta enfermedad de amanecer nostálgico
Que no se me pasa ni con la felicidad
Me tiene añorando algún contacto tántrico
Con esa mirada de infinita locura

Creo que he recordado esta mañana trágica
Tardes de tu mano en ese puerto mágico
Don